# STimage

In [2]:
import stlearn as st
st.settings.set_figure_params(dpi=300)
from pathlib import Path
import matplotlib.pyplot as plt
import sys; import scanpy as sc
from typing import Optional, Union
from anndata import AnnData
import pandas as pd
from typing import Optional, Union
from anndata import AnnData
from tqdm import tqdm
import seaborn as sns
import numpy as np
import os; import sys
from sklearn.multioutput import MultiOutputClassifier
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_auc_score
from skimage.color import rgb2hed
%matplotlib inline
from tensorflow.keras.utils import to_categorical
from numpy import array, argmax, load
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import backend
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.preprocessing import image

from PIL import Image
import lime; from lime import lime_image

import skimage
from sklearn.preprocessing import MinMaxScaler
from skimage.segmentation import mark_boundaries, watershed
from skimage.color import rgb2hed
from skimage.feature import peak_local_max
from skimage.measure import label
import glob
from tensorflow.keras.preprocessing import image as image_fun
from sklearn.preprocessing import OneHotEncoder

import scipy as sp
from scipy import ndimage as ndi
from skimage.morphology import area_opening
import math; import copy
from sklearn.model_selection import StratifiedKFold
import joblib
from sklearn.metrics import f1_score

def tiling(
        adata: AnnData,
        out_path: Union[Path, str] = "./tiling",
        library_id: str = None,
        crop_size: int = 40,
        target_size: int = 299,
        verbose: bool = False,
        copy: bool = False,
) -> Optional[AnnData]:
    

    if library_id is None:
        library_id = list(adata.uns["spatial"].keys())[0]

    # Check the exist of out_path
    if not os.path.isdir(out_path):
        os.mkdir(out_path)

    image = adata.uns["spatial"][library_id]["images"][adata.uns["spatial"]["use_quality"]]
    if image.dtype == np.float32 or image.dtype == np.float64:
        image = (image * 255).astype(np.uint8)
    img_pillow = Image.fromarray(image)
    tile_names = []

    with tqdm(
            total=len(adata),
            desc="Tiling image",
            bar_format="{l_bar}{bar} [ time left: {remaining} ]",
    ) as pbar:
        for imagerow, imagecol in zip(adata.obs["imagerow"], adata.obs["imagecol"]):
            imagerow_down = imagerow - crop_size / 2
            imagerow_up = imagerow + crop_size / 2
            imagecol_left = imagecol - crop_size / 2
            imagecol_right = imagecol + crop_size / 2
            tile = img_pillow.crop(
                (imagecol_left, imagerow_down, imagecol_right, imagerow_up)
            )
            # tile.thumbnail((target_size, target_size), Image.ANTIALIAS)
            tile = tile.resize((target_size, target_size))
            tile_name = library_id + "-" + str(imagecol) + "-" + str(imagerow) + "-" + str(crop_size)#np.arange(len(pd.Series(adata))+1).astype(str).str.zfill(4)+1 + "-" +
            out_tile = Path(out_path) / (tile_name + ".jpeg")
            tile_names.append(str(out_tile))
            if verbose:
                print(
                    "generate tile at location ({}, {})".format(
                        str(imagecol), str(imagerow)
                    )
                )
            tile.save(out_tile, "JPEG")

            pbar.update(1)

    adata.obs["tile_path"] = tile_names
    return adata if copy else None

#os.chdir("C:/Users/ONKAR")
#from STimage.stimage._utils import gene_plot

# 10X

In [3]:
BASE_PATH = Path("D:/Onkar_D/UQ/Project_Spt.Transcriptomics/Output_files")
TILE_PATH = BASE_PATH / "tiles"
TILE_PATH.mkdir(parents=True, exist_ok=True)


SAMPLE = "block1"
Sample1 = st.Read10X(BASE_PATH / SAMPLE, 
                  library_id=SAMPLE, 
                  count_file="V1_Breast_Cancer_Block_A_Section_1_filtered_feature_bc_matrix.h5",
                  quality="fulres",)
img = plt.imread(BASE_PATH / SAMPLE /"V1_Breast_Cancer_Block_A_Section_1_image.tif", 0)
Sample1.uns["spatial"][SAMPLE]['images']["fulres"] = img


SAMPLE = "block2"
Sample2 = st.Read10X(BASE_PATH / SAMPLE, 
                  library_id=SAMPLE, 
                  count_file="V1_Breast_Cancer_Block_A_Section_2_filtered_feature_bc_matrix.h5",
                  quality="fulres",)
                  #source_image_path=BASE_PATH / SAMPLE /"V1_Breast_Cancer_Block_A_Section_1_image.tif")
img = plt.imread(BASE_PATH / SAMPLE /"V1_Breast_Cancer_Block_A_Section_2_image.tif", 0)
Sample2.uns["spatial"][SAMPLE]['images']["fulres"] = img


for adata in [Sample1,Sample2,]:
    #Preprocessing
    st.pp.filter_genes(adata,min_cells=3)
    st.pp.log1p(adata)
    sc.pp.filter_cells(adata,min_counts=100)
    
    TILE_PATH_ = TILE_PATH / list(adata.uns["spatial"].keys())[0]
    TILE_PATH_.mkdir(parents=True, exist_ok=True)
    #tiling(adata, TILE_PATH_, crop_size=299)
    #st.pp.extract_feature(adata)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Log transformation step is finished in adata.X
Log transformation step is finished in adata.X


# FFPE

In [3]:
BASE_PATH = Path("D:/Onkar_D/UQ/Project_Spt.Transcriptomics/Output_files")
TILE_PATH = BASE_PATH / "tiles"
TILE_PATH.mkdir(parents=True, exist_ok=True)


SAMPLE = "FFPE_Breast_Cancer"
Sample3 = st.Read10X(BASE_PATH / SAMPLE, 
                  library_id=SAMPLE, 
                  count_file="Visium_FFPE_Human_Breast_Cancer_filtered_feature_bc_matrix.h5",
                  quality="fulres",)
img = plt.imread(BASE_PATH / SAMPLE /"Visium_FFPE_Human_Breast_Cancer_image.tif", 0)
Sample3.uns["spatial"][SAMPLE]['images']["fulres"] = img


Sample3_un_norm = copy.copy(Sample3)
for adata in [Sample3,]:
     
    st.pp.filter_genes(adata,min_cells=3)
    sc.pp.filter_cells(adata,min_counts=100)
    st.pp.log1p(adata)
    
    TILE_PATH_ = TILE_PATH / list(adata.uns["spatial"].keys())[0]
    TILE_PATH_.mkdir(parents=True, exist_ok=True)
    #tiling(adata, TILE_PATH_, crop_size=299)
    #st.pp.extract_feature(adata)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Log transformation step is finished in adata.X


### st.pp.extract_features will give us ResNet50 features

In [ ]:
wd = "D:/Onkar_D/UQ/Project_Spt.Transcriptomics/Output_files/pickle"
pd.DataFrame(Sample1.obsm['X_tile_feature']).to_csv(wd+'Sample1_small_ResNet50.csv')
pd.DataFrame(Sample2.obsm['X_tile_feature']).to_csv(wd+'Sample2_small_ResNet50.csv')

In [ ]:
wd = "D:/Onkar_D/UQ/Project_Spt.Transcriptomics/Output_files/pickle"
pd.DataFrame(Sample3.obsm['X_tile_feature']).to_csv(wd+'Sample3_small_ResNet50.csv')

# Concatenation of 10X & FFPE ResNet Features of Small and Big Tiles

### X_train -> 10X Tissue 1A -> Training Features both Small and Big
### X_test -> 10X Tissue 2A -> External Test Features both Small and Big
### X_test_ffpe -> FFPE -> External Test Features both Small and Big

In [4]:
wd = "D:/Onkar_D/UQ/Project_Spt.Transcriptomics/Output_files/pickle/"

#10X
Sample1_resnet = pd.read_csv(wd+'Sample1_small_ResNet50.csv').iloc[:,1:] #Renset Features of Small 299 size tiles adata.obsm[X_tile_feature] --> saved as dataframe
Sample1_big_resnet = pd.read_csv(wd+'Sample1_big_ResNet50.csv').iloc[:,1:] # Renset Features of Small 1950 size tiles adata.obsm[X_tile_feature] --> saved as dataframe
X_train = pd.concat([Sample1_resnet, Sample1_big_resnet],axis=1)
X_train.columns = np.arange(len(X_train.columns))

#10X
Sample2_resnet = pd.read_csv(wd+'Sample2_small_ResNet50.csv').iloc[:,1:] #Renset Features of Small 299 size tiles adata.obsm[X_tile_feature] --> saved as dataframe
Sample2_big_resnet = pd.read_csv(wd+'Sample2_big_ResNet50.csv').iloc[:,1:]
X_test = pd.concat([Sample2_resnet, Sample2_big_resnet],axis=1)
X_test.columns = np.arange(len(X_test.columns))

#FFPE
Sample2_resnet_ffpe = pd.read_csv(wd+'Sample3_small_ResNet50.csv').iloc[:,1:] #Renset Features of Small 299 size tiles adata.obsm[X_tile_feature] --> saved as dataframe
Sample2_big_resnet_ffpe = pd.read_csv(wd+'Sample3_big_ResNet50.csv').iloc[:,1:]
X_test_ffpe = pd.concat([Sample2_resnet_ffpe, Sample2_big_resnet_ffpe],axis=1)
X_test_ffpe.columns = np.arange(len(X_test_ffpe.columns))

print(X_train.shape, X_test.shape, X_test_ffpe.shape)

(3798, 4096) (3986, 4096) (2518, 4096)


# Hierarchial Clustering of the Spots 
### This information will be added along with Classifier to train regression

In [6]:
from sklearn.cluster import AgglomerativeClustering
hc1 = AgglomerativeClustering(n_clusters = 5, affinity = 'euclidean', linkage = 'ward')
hc2 = AgglomerativeClustering(n_clusters = 3, affinity = 'euclidean', linkage = 'ward')
hc3 = AgglomerativeClustering(n_clusters = 3, affinity = 'euclidean', linkage = 'ward')

Spots1_hc = hc1.fit_predict(X_train)
Spots2_hc = hc2.fit_predict(X_test)
Spots3_hc = hc3.fit_predict(X_test_ffpe)

## Train MLP Classifier for Concatenated Features

In [39]:
from sklearn.multioutput import MultiOutputClassifier; from sklearn.multioutput import MultiOutputRegressor; import lightgbm as lgb
import pandas as pd; import shap; import numpy as np; import joblib
from sklearn import preprocessing; from sklearn.preprocessing import LabelEncoder;
from sklearn import preprocessing; from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import roc_auc_score; from sklearn.model_selection import train_test_split; from sklearn.neural_network import MLPClassifier; from sklearn.neural_network import MLPRegressor

biomarker_list_1 = ["COX6C", "TTLL12", "CD74"]#['COX6C','TTLL12','PGM5','KRT5','SLITRK6', 'CPB1', "PABPC1", "GNAS", "HSP90AB1", "TFF3", "ATP1A1", "B2M", "FASN", "ESR1", "SPARC", "ACTG1", "HLA-B", "LINC00645", "MALAT1"]
biomarker_list_2 = ['COX6C','TTLL12', "PABPC1", "GNAS", "HSP90AB1", "TFF3", "ATP1A1", "B2M", "FASN", "SPARC"]

#

#lgb.LGBMRegressor()
model_c = MLPClassifier(max_iter=10000)
model_r = MLPRegressor(max_iter=10000)
    
MinMax_scaler_y_1 = preprocessing.MinMaxScaler(feature_range =(0, 1))
MinMax_scaler_y_2 = preprocessing.MinMaxScaler(feature_range =(0, 1))

Y_1 = Sample1.to_df()[biomarker_list_1]
Y_1 = MinMax_scaler_y_1.fit_transform(Y_1) 
Y_1 = pd.DataFrame(data=Y_1)
Y_1 = Y_1.apply(lambda x: pd.cut(x, bins=[-0.01, 0.5, 1.01], labels = [0, 1]))
#Y_1 = Y_1.apply(lambda x: pd.qcut(x, 2,duplicates='drop',labels=False))"""

test_Y_1 = Sample2.to_df()[biomarker_list_1]
test_Y_1 = MinMax_scaler_y_2.fit_transform(test_Y_1)
test_Y_1 = pd.DataFrame(data=test_Y_1)
test_Y_1 = test_Y_1.apply(lambda x: pd.cut(x, bins=[-0.01, 0.5, 1.01], labels = [0, 1]))
#test_Y_1 = test_Y_1.apply(lambda x: pd.qcut(x, 2,duplicates='drop',labels=False))
test_Y_1 = test_Y_1.fillna(0)

Y_1 = Y_1.astype('int64')
test_Y_1 = test_Y_1.astype('int64')

In [ ]:
import joblib; wd = "D:/Onkar_D/UQ/Project_Spt.Transcriptomics/Output_files/pickle/"

model_c = MultiOutputClassifier(model_c).fit(X_train, Y_1)
pred = model_c.predict(X_test)
joblib.dump(model_c, wd+'Temp-Biomarker_train.pkl')

# Regression 
## Added information along with concatenated Resnet features -> Hierarchial cluster outputs and Low or High Spot Classification output

In [9]:
Reg_train = pd.DataFrame()
Reg_train = Y_1
Reg_train["Spots1_hc"] = Spots1_hc


Reg_test = pd.DataFrame()
Reg_test = pd.DataFrame(pred)
Reg_test["Spots2_hc"] = Spots2_hc
Reg_test

,0,1,2,3,4,5,6,7,8,9,Spots2_hc
0,1,0,1,1,1,1,0,1,1,1,2
1,0,0,1,1,0,0,0,1,0,0,0
2,0,0,1,1,1,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2513,0,0,0,1,0,1,0,0,0,0,0
2514,0,0,0,1,0,1,0,0,0,0,0
2515,0,0,0,0,0,1,0,0,0,0,1
2516,0,0,0,1,0,1,0,0,0,0,0


## For every gene the particular Outputs of Low and High spot is selected and concatenated with resnet features and hierarchial cluster 

In [10]:
reg_pred = []
for i in range(0,len(pred[0])):
    Train = pd.concat([X_train, Reg_train.iloc[:,[i,len(pred[0])]]], axis=1)
    Test = pd.concat([X_test_ffpe, Reg_test.iloc[:,[i,len(pred[0])]]], axis=1)
    reg = model_r.fit(Train, Sample1.to_df()[biomarker_list_1[i]])
    pred2 = reg.predict(Test)
    reg_pred.append(pred2)
pd.DataFrame(reg_pred).T.to_csv(wd+"Regression_FFPE.csv", index = False)

# Can be ignored from below

In [4]:
wd = 'D:/Onkar_D/UQ/Project_Spt.Transcriptomics/Output_files/pickle/'
pred2_ffpe = pd.read_csv(wd+'Regression_FFPE.csv')
pred2_10x = pd.read_csv(wd+'Regression_10X.csv')

In [12]:
Train = pd.concat([X_train, Reg_train], axis=1)
Test = pd.concat([X_test, Reg_test], axis=1)

reg = MultiOutputRegressor(model_r).fit(Train, Sample1.to_df()[biomarker_list_1].values)
pred2 = reg.predict(Test)
joblib.dump(reg, wd+'New_model_reg_biomarker_FFPE.pkl')

['D:/Onkar_D/UQ/Project_Spt.Transcriptomics/Output_files/pickle/New_model_reg_biomarker_FFPE.pkl']